In [1]:
from agent_module.llm_request import send_request_choco
from agent_module.Gigachat import GigaChatBOT

gigachat = GigaChatBOT()

PROMPT_CLASSIFICATION = """Определи, является ли этот текст про запрос про кофе.
                           Пользователь должен задавать вопрос по название кофе или просто про кофе.
                           Вывод сделай в формате (строго следуй формату, не пиши ничего лишнего):
                           Является тематикаой: да или нет"""

PROMPT_DIALOGUE = """
Проанализируй текст сообщения пользователя и определи, хочет ли он продолжать диалог. 
Оцени его тон, интерес, вопросы, намеки на будущее общение и любые признаки отказа или закрытия темы.
Не опирайся только на конкретные слова "да" или "нет", учитывай контекст всего сообщения.
Если пользователь проявляет интерес, задает уточняющие вопросы или выражает готовность к дальнейшему обсуждению, то он хочет продолжать диалог. 
Если он выражает отказ, завершает общение или не проявляет интереса к дальнейшим контактам, то он не хочет продолжать диалог.

Вывод сделай в формате (строго следуй формату, не пиши ничего лишнего):
Хочет продолжать диалог: да или нет
"""

PROMPT_DETECTION = """
Проанализируй текст сообщения пользователя и определи, указал ли он один из следующих контактов:
1. Электронную почту
2. Номер телефона
3. Ссылку на Instagram
4. Ссылку на Telegram

Если контакт указан, выведи его. Если контакт не указан, напиши "нет".

Вывод сделай в формате (строго следуй формату):
Контакт: указанный контакт или нет
"""
PROMT_QUESTION_COMPANY = """
            Тебе нужно определить, интересуется ли человек информацией о компании или процессами, связанными с ее работой.
            Требуется выявить только те вопросы, которые напрямую касаются компании.
            Результат должен быть представлен в формате(Строго следуй формату вывода):
            Company ask: да или нет.
        """

PROMPT_FOR_GENERATION = """Ты эксперт по продажам. Ты должен продать товар пользователю.
                           На основе отзывов и запроса пользователя сгенерируй красивый ответ.
                           Ответ должен понравится пользоватлю, отобрадать, то что просил пользователь.
                           Генерируй только текст без различных добавлений и галюцинаций.
                           Пиши текст от лица первого лица как крутой эксперт по продажам.
                           """

In [2]:

class AgentBot:

    def __init__(self) -> None:
        pass

    def check_coffe_dialog(self, answer):

        get_answer = {"да": 0, "нет": 0} 

        for i in range(0, 3):
            
            #topic = send_request_choco(PROMPT_CLASSIFICATION, answer)
            topic = gigachat.query_text(PROMPT_CLASSIFICATION, answer)

            if "да" in topic.lower():
                get_answer["да"] += 1
            else:
                get_answer["нет"] += 1


        if get_answer["да"] > get_answer["нет"]:
            result = "да"
        elif get_answer["нет"] > get_answer["да"]:
            result = "нет"

        return result 


    def check_dialog_contionue(self, answer):

        get_answer_dialog = {"да": 0, "нет": 0} 

        for i in range(0, 3):
            #topic_dialog = send_request_choco(PROMPT_DIALOGUE, answer)
            topic_dialog = gigachat.query_text(PROMPT_DIALOGUE, answer)

            if "да" in topic_dialog.lower():
                get_answer_dialog["да"] += 1
            else:
                get_answer_dialog["нет"] += 1


        if get_answer_dialog["да"] > get_answer_dialog["нет"]:
            result_dialog = "да"
        elif get_answer_dialog["нет"] > get_answer_dialog["да"]:
            result_dialog = "нет"

        return result_dialog

    def retrieve_contact(self, answer):

        get_answer_contact = {"contact": 0, "нет": 0} 

        for i in range(0, 3):
            contact = gigachat.query_text(PROMPT_DETECTION, answer)

            if "нет" in contact.lower():
                get_answer_contact["нет"] += 1
            else:
                get_answer_contact["contact"] += 1

            if get_answer_contact["contact"] > get_answer_contact["нет"]:
                result_contact = contact.split(":")[1].split()
                
            elif get_answer_contact["нет"] > get_answer_contact["contact"]:
                result_contact = "нет"

            return result_contact

    def company_ask_detection(self, answer):

        get_answer_company = {"да": 0, "нет": 0} 

        for i in range(0, 3):
            #contact = send_request_choco(PROMT_QUESTION_COMPANY, answer)
            contact = gigachat.query_text(PROMT_QUESTION_COMPANY, answer)
            if "нет" in contact.lower():
                get_answer_company["нет"] += 1
            else:
                get_answer_company["да"] += 1
            if get_answer_company["да"] > get_answer_company["нет"]:
                result_contact = contact.split(":")[1].split()
            elif get_answer_company["нет"] > get_answer_company["да"]:
                result_contact = "нет"

            return result_contact

In [27]:
answer = """Добрый день Подскажите, пожалуйста, откуда у вас мой контакт?"""

company_ask_detection(answer)

['да']

In [7]:
from DB.db_request import send_query
from agent_module.llm_request import send_request_choco

answer = """Расскажи мне про вкусное кофе"""

cof = AgentBot().check_coffe_dialog(answer)

if cof == "да":
    #mod = send_request(PROMT_FOR_GET_TOP_VIEWS, answer)
    mod = gigachat.query_text(PROMT_FOR_GET_TOP_VIEWS, answer)
    retr = send_query(answer, 5)
    USER_PROMPT = f"""Запрос пользователя: {answer}
                      Отзывы: {retr}"""
    #otvet = send_request(PROMPT_FOR_GENERATION, USER_PROMPT)
    otvet = gigachat.query_text(PROMPT_FOR_GENERATION, USER_PROMPT)
    print(otvet)
else:
    res_dial = AgentBot().check_dialog_contionue(answer)
    contact = AgentBot().retrieve_contact(answer)
    if contact.lower() != "нет":
        
    print(res_dial)
    print(contact)


Привет! Я рад, что ты интересуешься кофе. Судя по отзывам, которые ты предоставил, кофе "Бэрри" и "Натти" пользуются большой популярностью среди покупателей. В отзывах часто упоминается отличный вкус и качество этих марок. Если ты решишь попробовать кофе "Бэрри", то я бы рекомендовал начать с вкуса "Очень хороший кофе". А если выберешь "Натти", то обрати внимание на отзывы о вкусе "Отличный кофе". Надеюсь, мой совет поможет тебе выбрать вкусный кофе!


In [18]:
import requests
import json
#напишите мне пожалуйста на popset26@gmail.com
# Какие самые негативные темы обсуждают пользовтаели
# "напишите мне пожалуйста на asck.petrov@yandex.ru"
# удалите мой контакт из вашей рассылки
# "Расскажи мне про вкусный кофе"
# URL вашего API
url = "http://localhost:8002/query"

# Данные, которые мы отправляем в запросе
data = {
    "answer": "напишите мне пожалуйста на popset26@gmail.com"
}

# Отправляем POST-запрос
response = requests.post(url, json=data)

# Проверяем, успешен ли запрос
if response.status_code == 200:
    # Если успешен, выводим результат
    result = response.json()
    print("Результат:", result['result'])
else:
    # Если произошла ошибка, выводим сообщение
    print(f"Ошибка {response.status_code}: {response.text}")



Результат: Письмо отправлено!


In [ ]:
# в ответах учитывать конкретные числа (упоминаний и тд)